___

<a href='http://www.pieriandata.com'> <img src='../Pierian_Data_Logo.png' /></a>
___

# Non-Negative Matric Factorization

Let's repeat thet opic modeling task from the previous lecture, but this time, we will use NMF instead of LDA.

## Data

We will be using articles scraped from NPR (National Public Radio), obtained from their website [www.npr.org](http://www.npr.org)

In [49]:
import pandas as pd

In [50]:
npr = pd.read_csv('/content/npr.csv', encoding='utf-8')

In [51]:
npr.head()

,Article
0,"In the Washington of 2016, even when the polic..."
1,Donald Trump has used Twitter — his prefe...
2,Donald Trump is unabashedly praising Russian...
3,"Updated at 2:50 p. m. ET, Russian President Vl..."
4,"From photography, illustration and video, to d..."


Notice how we don't have the topic of the articles! Let's use LDA to attempt to figure out clusters of the articles.

## Preprocessing

In [52]:
from sklearn.feature_extraction.text import TfidfVectorizer

**`max_df`**` : float in range [0.0, 1.0] or int, default=1.0`<br>
When building the vocabulary ignore terms that have a document frequency strictly higher than the given threshold (corpus-specific stop words). If float, the parameter represents a proportion of documents, integer absolute counts. This parameter is ignored if vocabulary is not None.

**`min_df`**` : float in range [0.0, 1.0] or int, default=1`<br>
When building the vocabulary ignore terms that have a document frequency strictly lower than the given threshold. This value is also called cut-off in the literature. If float, the parameter represents a proportion of documents, integer absolute counts. This parameter is ignored if vocabulary is not None.

In [53]:
tfidf = TfidfVectorizer(max_df=0.95, min_df=2, stop_words='english')

In [54]:
dtm = tfidf.fit_transform(npr['Article'])

In [55]:
dtm

<11992x54777 sparse matrix of type '<class 'numpy.float64'>'
	with 3033388 stored elements in Compressed Sparse Row format>

## NMF

In [56]:
from sklearn.decomposition import NMF

In [57]:
nmf_model = NMF(n_components=7,random_state=42)

In [58]:
# This can take awhile, we're dealing with a large amount of documents!
nmf_model.fit(dtm)

NMF(n_components=7, random_state=42)

## Displaying Topics

In [62]:
len(tfidf.get_feature_names_out())

54777

In [63]:
import random

In [64]:
for i in range(10):
    random_word_id = random.randint(0,54776)
    print(tfidf.get_feature_names_out()[random_word_id])

carte
paranormal
yahya
nkurunziza
beachy
funereal
rejuvenate
muppet
sandman
gdansk


In [65]:
for i in range(10):
    random_word_id = random.randint(0,54776)
    print(tfidf.get_feature_names_out()[random_word_id])

obstetrics
nassirian
julián
hardness
oftentimes
deacons
mccaslin
unsurprisingly
running
multivalent


In [ ]:
len(nmf_model.components_)

In [ ]:
nmf_model.components_

In [ ]:
len(nmf_model.components_[0])

In [ ]:
single_topic = nmf_model.components_[0]

In [ ]:
# Returns the indices that would sort this array.
single_topic.argsort()

In [ ]:
# Word least representative of this topic
single_topic[18302]

In [ ]:
# Word most representative of this topic
single_topic[42993]

In [ ]:
# Top 10 words for this topic:
single_topic.argsort()[-10:]

In [ ]:
top_word_indices = single_topic.argsort()[-10:]

In [ ]:
for index in top_word_indices:
    print(tfidf.get_feature_names_out()[index])

These look like business articles perhaps... Let's confirm by using .transform() on our vectorized articles to attach a label number. But first, let's view all the 10 topics found.

In [66]:
for index,topic in enumerate(nmf_model.components_):
    print(f'THE TOP 15 WORDS FOR TOPIC #{index}')
    print([tfidf.get_feature_names_out()[i] for i in topic.argsort()[-15:]])
    print('\n')

THE TOP 15 WORDS FOR TOPIC #0
['new', 'research', 'like', 'patients', 'health', 'disease', 'percent', 'women', 'virus', 'study', 'water', 'food', 'people', 'zika', 'says']


THE TOP 15 WORDS FOR TOPIC #1
['gop', 'pence', 'presidential', 'russia', 'administration', 'election', 'republican', 'obama', 'white', 'house', 'donald', 'campaign', 'said', 'president', 'trump']


THE TOP 15 WORDS FOR TOPIC #2
['senate', 'house', 'people', 'act', 'law', 'tax', 'plan', 'republicans', 'affordable', 'obamacare', 'coverage', 'medicaid', 'insurance', 'care', 'health']


THE TOP 15 WORDS FOR TOPIC #3
['officers', 'syria', 'security', 'department', 'law', 'isis', 'russia', 'government', 'state', 'attack', 'president', 'reports', 'court', 'said', 'police']


THE TOP 15 WORDS FOR TOPIC #4
['primary', 'cruz', 'election', 'democrats', 'percent', 'party', 'delegates', 'vote', 'state', 'democratic', 'hillary', 'campaign', 'voters', 'sanders', 'clinton']


THE TOP 15 WORDS FOR TOPIC #5
['love', 've', 'don', 'al

### Attaching Discovered Topic Labels to Original Articles

In [ ]:
dtm

In [ ]:
dtm.shape

In [ ]:
len(npr)

In [67]:
topic_results = nmf_model.transform(dtm)

In [ ]:
topic_results.shape

In [ ]:
topic_results[0]

In [ ]:
topic_results[0].round(2)

In [ ]:
topic_results[0].argmax()

This means that our model thinks that the first article belongs to topic #1.

### Combining with Original Data

In [ ]:
npr.head()

In [ ]:
topic_results.argmax(axis=1)

In [68]:
npr['Topic'] = topic_results.argmax(npaxis=1)

In [70]:
npr.head(10)

,Article,Topic
0,"In the Washington of 2016, even when the polic...",1
1,Donald Trump has used Twitter — his prefe...,1
2,Donald Trump is unabashedly praising Russian...,1
3,"Updated at 2:50 p. m. ET, Russian President Vl...",3
4,"From photography, illustration and video, to d...",6
5,I did not want to join yoga class. I hated tho...,5
6,With a who has publicly supported the debunk...,0
7,"I was standing by the airport exit, debating w...",0
8,"If movies were trying to be more realistic, pe...",0
9,"Eighteen years ago, on New Year’s Eve, David F...",5


In [72]:
mytopic_dict = {0:'health',1:'election',2:'legis',3:'policy',4:'candidates',5:'music',6:'edu'}
npr['Topic Label'] = npr['Topic'].map(mytopic_dict)
npr.head()

,Article,Topic,Topic Label
0,"In the Washington of 2016, even when the polic...",1,election
1,Donald Trump has used Twitter — his prefe...,1,election
2,Donald Trump is unabashedly praising Russian...,1,election
3,"Updated at 2:50 p. m. ET, Russian President Vl...",3,policy
4,"From photography, illustration and video, to d...",6,edu


## Great work!